In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
import numpy as np
from datetime import datetime
import time
import seaborn

In [2]:
os.chdir(r'C:\Users\USER\Desktop\Dynamic pricing')

product = pd.read_csv('pruduct.csv', encoding='utf-8')
session = pd.read_csv('Session.csv', encoding='utf-8')
search1 = pd.read_csv('Search1.csv', encoding='utf-8')
search2 = pd.read_csv('Search2.csv', encoding='utf-8')
custom =  pd.read_csv('custom.csv', encoding='utf-8')
master =  pd.read_csv('Master.csv', encoding='utf-8')

C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# 브랜드 변수에서 특수문자 제거
product['PD_BRA_NM'] = list(map(lambda x:re.sub("[[,\](,)\s]", "", x), product['PD_BRA_NM']))

# 구매가격 변수를 str -> int 변환.
product['PD_BUY_AM'] = list(map(lambda x:x.replace(",",""),product['PD_BUY_AM']))
product['PD_BUY_AM'] = product['PD_BUY_AM'].astype(int)

# 구매개수 변수를 str&int -> int로 변환.
product['PD_BUY_CT'] = product['PD_BUY_CT'].astype(str)
product['PD_BUY_CT'] = list(map(lambda x:x.replace(",",""),product['PD_BUY_CT']))
product['PD_BUY_CT'] = product['PD_BUY_CT'].astype(int)

# product를 CLNT_ID 순으로 정렬
product = product.sort_values(by=['CLNT_ID'], axis=0)

## product에 새로운 열 "TOT_AM" 생성 (PD_BUY_AM는 제품 하나 당 개수이므로, 이를 구매한 제품의 갯수와 곱한 "총 지출 금액"이 "TOT_AM"임)
product["TOT_AM"] = product["PD_BUY_AM"] * product["PD_BUY_CT"]

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Possible nested set at position 1
  


In [6]:
session = session.sort_values(by=['CLNT_ID', 'SESS_ID'], axis=0)
product = product.sort_values(by=['CLNT_ID', 'SESS_ID'], axis=0)

In [7]:
raw = pd.merge(session, product, how='left', on = ['CLNT_ID', 'SESS_ID'])

In [9]:
# mean_buy_am: CLNT_ID와 SESS_ID가 모두 같은 행들을 "TOT_AM"에 대해 합계를 구한 것
tot_buy_am = product.groupby(['CLNT_ID', 'SESS_ID'])['TOT_AM'].agg('sum').tolist()

In [10]:
# mean_buy_am: CLNT_ID와 SESS_ID가 모두 같은 행들을 "TOT_AM"에 대해 평균를 구한 것
mean_buy_am = product.groupby(['CLNT_ID', 'SESS_ID'])['TOT_AM'].agg('mean').tolist()

In [11]:
# std_buy_am: CLNT_ID와 SESS_ID가 모두 같은 행들을 "TOT_AM"에 대해 표준편차를 구한 것
std_buy_am = product.groupby(['CLNT_ID', 'SESS_ID'])['TOT_AM'].agg('std').tolist()

In [12]:
# tot_buy_ct: CLNT_ID와 SESS_ID가 모두 같은 행들을 "PD_BUY_CT"에 대해 총합를 구한 것
tot_buy_ct = product.groupby(['CLNT_ID', 'SESS_ID'])['PD_BUY_CT'].agg('sum').tolist()

# mean_buy_ct: CLNT_ID와 SESS_ID가 모두 같은 행들을 "PD_BUY_CT"에 대해 평균를 구한 것
mean_buy_ct = product.groupby(['CLNT_ID', 'SESS_ID'])['PD_BUY_CT'].agg('mean').tolist()

# std_buy_ct: CLNT_ID와 SESS_ID가 모두 같은 행들을 "PD_BUY_CT"에 대해 평균를 구한 것
std_buy_ct = product.groupby(['CLNT_ID', 'SESS_ID'])['PD_BUY_CT'].agg('std').tolist()

In [13]:
# tot_prod_price: CLNT_ID와 SESS_ID가 모두 같은 행들을 "PD_BUY_AM"에 대해 총합를 구한 것
tot_prod_price = product.groupby(['CLNT_ID', 'SESS_ID'])['PD_BUY_AM'].agg('sum').tolist()

# mean_prod_price: CLNT_ID와 SESS_ID가 모두 같은 행들을 "PD_BUY_AM"에 대해 평균를 구한 것
mean_prod_price = product.groupby(['CLNT_ID', 'SESS_ID'])['PD_BUY_AM'].agg('mean').tolist()

# std_prod_price: CLNT_ID와 SESS_ID가 모두 같은 행들을 "PD_BUY_AM"에 대해 평균를 구한 것
std_prod_price = product.groupby(['CLNT_ID', 'SESS_ID'])['PD_BUY_AM'].agg('std').tolist()

In [15]:
session["TOT_BUY_AM"] = tot_buy_am
session["MEAN_BUY_AM"] = mean_buy_am
session["STD_BUY_AM"] = std_buy_am
session["TOT_BUY_CT"] = tot_buy_ct
session["MEAN_BUY_CT"] = mean_buy_ct
session["STD_BUY_CT"] = std_buy_ct
session["TOT_PROD_PRICE"] = tot_prod_price
session["TOT_PROD_PRICE"] = mean_prod_price
session["TOT_PROD_PRICE"] = std_prod_price

In [16]:
session.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,BUY_TOT_AM,TOT_BUY_AM,MEAN_BUY_AM,STD_BUY_AM,TOT_BUY_CT,MEAN_BUY_CT,STD_BUY_CT,TOT_PROD_PRICE
172344,8,6964877,7,20180606,38.0,366,mobile,Gyeonggi-do,Gwangmyeong-si,38000,38000,38000.0,NaN,1,1.0,NaN,NaN
1398652,8,7017688,2,20180605,55.0,468,mobile,Seoul,Seoul,38000,38000,38000.0,NaN,1,1.0,NaN,NaN
1435369,14,1156991,5,20180911,256.0,"7,098",mobile,Gyeonggi-do,Ansan-si,69800,69800,34900.0,7071.067812,2,1.0,0.0,7071.067812
142972,25,5317297,213,20180703,33.0,865,desktop,Gyeonggi-do,Hwaseong-si,39800,39800,39800.0,NaN,1,1.0,NaN,NaN
142971,25,6059256,204,20180621,7.0,99,desktop,Gyeonggi-do,Osan-si,47000,47000,47000.0,NaN,1,1.0,NaN,NaN
